
# This is the BRDF and topographic correction using RossThick-LiSparse model (BRDF)  and SCS+c (Topo) models



In [1]:

#BRDF
%pip install numpy rasterio scipy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import rasterio
from rasterio.plot import show
from sklearn.linear_model import LinearRegression

# BRDF 
## RossThick-LiSparse model for BRDF correction

In [ ]:

# Example BRDF model function
def brdf_correction(surface_reflectance, sza, vza, raa, brdf_coeffs):
    """
    Perform BRDF correction using RossThick-LiSparse model.
    
    Args:
        surface_reflectance (np.array): Measured surface reflectance.
        sza (float): Solar zenith angle in degrees.
        vza (float): View zenith angle in degrees.
        raa (float): Relative azimuth angle in degrees.
        brdf_coeffs (dict): BRDF coefficients for isotropic, volumetric, and geometric kernels.
    
    Returns:
        np.array: BRDF-corrected reflectance.
    """
    # Convert angles to radians
    sza_rad = np.deg2rad(sza)
    vza_rad = np.deg2rad(vza)
    raa_rad = np.deg2rad(raa)
    
    # Compute BRDF kernels
    Kiso = 1  # Isotropic kernel
    Kvol = (np.pi / 2 - sza_rad) * np.cos(sza_rad) + np.sin(sza_rad) - \
           (np.pi / 2 - vza_rad) * np.cos(vza_rad) - np.sin(vza_rad)
    Kgeo = np.tan(sza_rad) * np.tan(vza_rad) * np.cos(raa_rad)
    
    # BRDF model
    brdf = (brdf_coeffs['kiso'] * Kiso + brdf_coeffs['kvol'] * Kvol + brdf_coeffs['kgeo'] * Kgeo)
    
    # Normalize to a standard solar and view geometry
    brdf_standard = brdf_coeffs['kiso']  # Assuming standard geometry is nadir
    
    # Correct reflectance
    corrected_reflectance = surface_reflectance * (brdf_standard / brdf)
    return corrected_reflectance

# Example surface reflectance and angles
surface_reflectance = np.array([[0.2, 0.3], [0.25, 0.35]])  # Reflectance values
sza, vza, raa = 30, 10, 45  # Solar zenith, view zenith, and relative azimuth angles

# BRDF coefficients (example values)
brdf_coeffs = {'kiso': 0.5, 'kvol': 0.2, 'kgeo': 0.3}

# Apply BRDF correction
corrected_reflectance = brdf_correction(surface_reflectance, sza, vza, raa, brdf_coeffs)
print("BRDF-Corrected Reflectance:\n", corrected_reflectance)

# FLEX correction (BRDF)


In [ ]:

def flec_correction(reflectance, sza, sza_ref=45):
    """
    Apply FLEC correction to surface reflectance data.
    
    Args:
        reflectance (np.array): Measured reflectance values (1D or 2D array).
        sza (np.array): Solar zenith angle (degrees), same size as reflectance.
        sza_ref (float): Reference solar zenith angle for correction (default: 45°).
    
    Returns:
        np.array: FLEC-corrected reflectance values.
    """
    # Ensure inputs are numpy arrays
    reflectance = np.array(reflectance)
    sza = np.array(sza)
    
    # Flatten arrays for linear regression if needed
    X = sza.flatten().reshape(-1, 1)  # SZA as independent variable
    Y = reflectance.flatten()         # Reflectance as dependent variable
    
    # Perform linear regression to estimate 'a'
    reg = LinearRegression()
    reg.fit(X, Y)
    a = reg.coef_[0]  # Slope of the regression line
    
    # Apply FLEC correction
    reflectance_corrected = reflectance + a * (sza_ref - sza)
    
    return reflectance_corrected, a


# Example data: Reflectance and SZA
reflectance = np.array([[0.2, 0.25], [0.3, 0.35]])  # Measured reflectance
sza = np.array([[30, 35], [40, 45]])  # Solar zenith angle (degrees)

# Apply FLEC correction
corrected_reflectance, slope_a = flec_correction(reflectance, sza)

print("Corrected Reflectance:\n", corrected_reflectance)
print("Regression Slope (a):", slope_a)


import rasterio

with rasterio.open("path/to/reflectance.tif") as reflectance_src:
    reflectance = reflectance_src.read(1)

with rasterio.open("path/to/sza.tif") as sza_src:
    sza = sza_src.read(1)


### how to run the code in python
corrected_reflectance, slope_a = flec_correction(reflectance, sza, sza_ref=45)


from rasterio.transform import from_origin

# Define transform and metadata for the output raster
transform = reflectance_src.transform
meta = reflectance_src.meta
meta.update(dtype='float32', count=1)

# Save the corrected reflectance
with rasterio.open("path/to/corrected_reflectance.tif", "w", **meta) as dst:
    dst.write(corrected_reflectance.astype('float32'), 1)





# Topo


In [ ]:


# SCS+C Topographic Correction
def scs_correction(surface_reflectance, slope, aspect, sza, saa, c_factor):
    """
    Perform SCS+C topographic correction.
    
    Args:
        surface_reflectance (np.array): Measured surface reflectance.
        slope (np.array): Terrain slope in degrees.
        aspect (np.array): Terrain aspect in degrees.
        sza (float): Solar zenith angle in degrees.
        saa (float): Solar azimuth angle in degrees.
        c_factor (float): Correction factor (calculated per band).
    
    Returns:
        np.array: Topographically corrected reflectance.
    """
    # Convert angles to radians
    slope_rad = np.deg2rad(slope)
    aspect_rad = np.deg2rad(aspect)
    sza_rad = np.deg2rad(sza)
    saa_rad = np.deg2rad(saa)
    
    # Illumination angle
    cos_i = np.cos(sza_rad) * np.cos(slope_rad) + \
            np.sin(sza_rad) * np.sin(slope_rad) * np.cos(saa_rad - aspect_rad)
    
    # Apply SCS+C correction
    corrected_reflectance = surface_reflectance * (np.cos(sza_rad) + c_factor) / (cos_i + c_factor)
    return corrected_reflectance


# Example raster data (simulated)
reflectance = np.array([[0.2, 0.25], [0.3, 0.35]])  # Surface reflectance
slope = np.array([[15, 20], [25, 30]])  # Slope in degrees
aspect = np.array([[90, 180], [270, 360]])  # Aspect in degrees
sza, saa = 30, 135  # Solar zenith and azimuth angles
c_factor = 0.1  # Correction factor (example value)

# Apply SCS+C correction
corrected_reflectance = scs_correction(reflectance, slope, aspect, sza, saa, c_factor)
print("Topographically Corrected Reflectance:\n", corrected_reflectance)



In [ ]:

#Combining BRDF and Topographic Corrections
#Apply the BRDF correction first, then the topographic correction:

# Step 1: BRDF correction
brdf_corrected = brdf_correction(reflectance, sza, vza, raa, brdf_coeffs)

# Step 2: Topographic correction
final_corrected = scs_correction(brdf_corrected, slope, aspect, sza, saa, c_factor)
print("Final Corrected Reflectance:\n", final_corrected)





In [ ]:
##Loading Real Data
# Use rasterio to load and process geospatial data for slope, aspect, and reflectance:

with rasterio.open("path/to/reflectance.tif") as src:
    reflectance = src.read(1)
with rasterio.open("path/to/slope.tif") as src:
    slope = src.read(1)
with rasterio.open("path/to/aspect.tif") as src:
    aspect = src.read(1)

final_corrected = scs_correction(brdf_corrected, slope, aspect, sza, saa, c_factor)
